<a href="https://colab.research.google.com/github/Zamoca42/TIL/blob/main/Python/Pyside_widget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 메신저 만들어보기

## ui design

<img width="399" alt="스크린샷 2023-01-25 오후 5 52 19" src="https://user-images.githubusercontent.com/96982072/214583838-8217fa89-74d9-4027-a081-a062f7abad1b.png">

- list view box
  - object name : `list_chat`

- line edit(입력 박스)
  - nickname_object : `edit_nickname`
  - chat_object : `edit_text`

- push button
  - object name : `btn_send`

### 실습코드

1. 위의 ui를 python 파일로 변환  
  ```
  pyside6-uic ui.ui > ui.py
  ```
2. `edit_text`에 내용 입력 시 메세지 박스로 출력

In [ ]:
#...

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.ui.btn_send.clicked.connect(self.send)
    
    def send(self):
        mb = QMessageBox()
        mb.setText(self.ui.edit_text.text())
        mb.exec()

#...

- 결과  

  ![스크린샷 2023-01-25 오후 7 37 38](https://user-images.githubusercontent.com/96982072/214542052-4952ea7c-7d60-4a29-a539-eb09a770586d.png)

## List View

- List View
  - model이 포함되지 않음
    - item과 model을 설정해주어야함

- Model 
  - 어플리케이션에서 사용되는 데이터와 그 데이터를 처리하는 부분
- View 
  - 사용자에서 보여지는 UI 부분
- Controller 
  - 사용자의 입력(Action)을 받고 처리하는 부분

- MVC, MVVM 패턴
  - https://beomy.tistory.com/43


### 실습코드2

1. `list_chat`에서 setModel
2. text를 입력하고 전송을 누르면 list_view에서 보여줌

In [ ]:
import sys
from PySide6.QtWidgets import QMainWindow, QApplication, QMessageBox
from ui import Ui_MainWindow # ui.py

from PySide6.QtGui import QStandardItemModel, QStandardItem

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.model = QStandardItemModel()
        self.ui.list_chat.setModel(self.model)

        self.ui.btn_send.clicked.connect(self.send)
    
    def send(self):
        text = self.ui.edit_text.text()
        item = QStandardItem(text)
        self.model.appendRow(item)

#...

- 결과  
  
  ![스크린샷 2023-01-25 오후 7 51 54](https://user-images.githubusercontent.com/96982072/214550593-36cc1c4a-cc81-4f0a-a74b-9955fbce32fa.png)


## List widget, line edit event

- list_view -> list widget으로 변경  

  <img width="529" alt="스크린샷 2023-01-25 오후 10 37 06" src="https://user-images.githubusercontent.com/96982072/214589519-b1484341-f835-4216-823b-ae8c74b370e8.png">

  - list widget은 model이 포함
    - 설정할 필요 없음

### 실습코드3


- list_view -> list widget으로 변경
- nickname 추가
- 전송버튼 대신 enter키 누르면 전송기능
  - `returnPressed.connect(self.send)`

In [ ]:
class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        #self.model = QStandardItemModel()
        #self.ui.list_chat.setModel(self.model)

        self.ui.btn_send.clicked.connect(self.send)
        self.ui.edit_text.returnPressed.connect(self.send)
    
    def send(self):
        nickname = self.ui.edit_nickname.text()
        text = self.ui.edit_text.text()
        # item = QStandardItem(text)
        # self.model.appendRow(item)
        self.ui.list_chat.addItem(f"{nickname}: {text}")
#...

- 결과  
  
  ![스크린샷 2023-01-25 오후 9 52 54](https://user-images.githubusercontent.com/96982072/214588253-5563b9d7-71a3-4391-88fd-077635d5e549.png)


## file

### 실습 코드

- 메신저을 위한 브로드캐스트 서버대신 파일을 사용
- 랜덤닉네임 생성해서 불러오기
- 채팅 내용을 전송후에는 비우기


In [ ]:
#...

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        #self.model = QStandardItemModel()
        #self.ui.list_chat.setModel(self.model)

        self.ui.btn_send.clicked.connect(self.send)
        self.ui.edit_text.returnPressed.connect(self.send)

        nickname = self.random_nickname()
        self.ui.edit_nickname.setText(nickname)
    
    def send(self):
        nickname = self.ui.edit_nickname.text()
        text = self.ui.edit_text.text()
        # item = QStandardItem(text)
        # self.model.appendRow(item)
        # self.ui.list_chat.addItem(f"{nickname}: {text}")
        msg = f"{nickname}: {text}"

        # 파일에다가 msg 쓰기
        with open("./server.txt", "a+") as f:
            f.writelines(msg)
        
        self.ui.edit_text.clear()
    
    def random_nickname(self):
        nickname = random.choice(["홍길동", "박보검", "한소희"])
        num = random.randint(1,1000)
        return f"{nickname}{num}"

#...

- 결과  
  
  ![스크린샷 2023-01-25 오후 10 14 40](https://user-images.githubusercontent.com/96982072/214594096-b24326d9-742d-4f11-9e01-5843f1695ac1.png)




## 여러명 채팅

- 창 여러개를 띄워 여러명과 채팅하는 상황 만들기
- 서버 파일 로드해보기
  - listen
  - server.txt에 저장된 메세지 리스트 박스로 불러오기
  - 한번 불러온 메세지는 다시 불러오지 않기
  - 처음 실행할 때 저장된 메세지 모두 불러오기

- 타이머 만들고 특정 시간마다 server.txt 불러오기
  - `QTimer()`

- 처음 실행 시 환영 메세지 추가

In [ ]:
from PySide6.QtCore import QTimer

# 타이머
self.timer = QTimer()
self.timer.setInterval(500)
self.timer.timeout.connect(self.listen)
self.timer.start()

### 실습코드

In [ ]:
class MainWindow(QMainWindow):
    last_read = 0

    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.ui.btn_send.clicked.connect(self.send)
        self.ui.edit_text.returnPressed.connect(self.send)

        nickname = self.random_nickname()
        self.ui.edit_nickname.setText(nickname)

        # 환영합니다 메세지
        with open("./server.txt", "a+") as f:
            f.writelines(f"---------{nickname}님이 입장하셨습니다\n")

        self.listen()

        self.timer = QTimer()
        self.timer.setInterval(500)
        self.timer.timeout.connect(self.listen)
        self.timer.start()
        
    def send(self):
        nickname = self.ui.edit_nickname.text()
        text = self.ui.edit_text.text()
        msg = f"{nickname}: {text}"

        # 파일에다가 msg 쓰기
        with open("./server.txt", "a+") as f:
            f.writelines(msg + "\n")
        
        self.ui.edit_text.clear()

        # # 읽어오기
        # self.listen()
    
    def random_nickname(self):
        nickname = random.choice(["홍길동", "박보검", "한소희"])
        num = random.randint(1,1000)
        return f"{nickname}{num}"
    
    def listen(self):
        try:
            with open("./server.txt", "r") as f:
                lines = f.readlines()
            lines = [x.strip() for x in lines]
            self.ui.list_chat.addItems(lines[self.last_read:])
            self.last_read = len(lines)
            self.ui.list_chat.scrollToBottom()
        except:
            pass

- 결과

  ![스크린샷 2023-01-26 오전 1 15 28](https://user-images.githubusercontent.com/96982072/214617579-6a9d503f-c695-4a92-b709-5f02cc78f1e3.png)


# 포스 만들어보기

## 디자인

## Table widget

- 현재 시각 표시
- 더미 데이터 만들고 주문 내역 가져오기
  - `QTableWidgetItem()`



In [ ]:
import sys
import datetime
from PySide6.QtWidgets import QMainWindow, QApplication, QTableWidgetItem
from PySide6.QtCore import QTimer

from ui import Ui_MainWindow # ui.py

dummy = [
    {"menu": "에스프레소 L", "quantity": "10", "order_amount": "10000", "time": "2021-07-22 09:14", "status": "waiting"},
    {"menu": "에스프레소 L", "quantity": "10", "order_amount": "10000", "time": "2021-07-22 09:14", "status": "waiting"}
]

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.timer = QTimer()
        self.timer.setInterval(990)
        self.timer.timeout.connect(self.tick)
        self.timer.start()

        self.load()

    def tick(self):
        now = datetime.datetime.now()
        self.ui.lb_now.setText(f"현재시각 : {now}")
    
    def load(self):
        for d in dummy:
            r = self.ui.table_orders.rowCount()
            self.ui.table_orders.insertRow(r)
            self.ui.table_orders.setItem(r, 0, QTableWidgetItem(d["menu"]))
            self.ui.table_orders.setItem(r, 1, QTableWidgetItem(d["quantity"]))
            self.ui.table_orders.setItem(r, 2, QTableWidgetItem(d["order_amount"]))
            self.ui.table_orders.setItem(r, 3, QTableWidgetItem(d["time"]))
            self.ui.table_orders.setItem(r, 4, QTableWidgetItem(d["status"]))



if __name__ == "__main__":
    app = QApplication()

    window = MainWindow()
    window.show()

    sys.exit(app.exec())

## 주문 금액 계산

- lcd 주문 업데이트
- 더미 데이터를 주문데이터로 바꾸기
- 주문 넣기탭
  - 메뉴별 금액
  - 사이즈
  - 주문 수량
  - 총 주문 수량

In [ ]:
orders = [
    {"menu": "에스프레소 L", "quantity": "10", "order_amount": "10000", "time": "2021-07-22 09:14", "status": "waiting"},
    {"menu": "에스프레소 L", "quantity": "10", "order_amount": "10000", "time": "2021-07-22 09:14", "status": "complete"}
]

menu_widegets = [
    "radio_espresso",
    "radio_americano",
    "radio_latte",
    "radio_mocca",
    "radio_choco_smoothie",
    "radio_berry_smoothie"
]

size_widegets = [
    "radio_size_s",
    "radio_size_m",
    "radio_size_l",
    "radio_size_xl"
]

class MainWindow(QMainWindow):
    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.timer = QTimer()
        self.timer.setInterval(990)
        self.timer.timeout.connect(self.tick)
        self.timer.start()

        self.load()

        for w in menu_widegets + size_widegets:
            widget = getattr(self.ui, w)
            widget.clicked.connect(self.calc)

        self.calc()

    def tick(self):
        now = datetime.datetime.now()
        self.ui.lb_now.setText(f"현재시각 : {now}")
    
    def load(self):
        for d in orders:
            r = self.ui.table_orders.rowCount()
            self.ui.table_orders.insertRow(r)
            self.ui.table_orders.setItem(r, 0, QTableWidgetItem(d["menu"]))
            self.ui.table_orders.setItem(r, 1, QTableWidgetItem(d["quantity"]))
            self.ui.table_orders.setItem(r, 2, QTableWidgetItem(d["order_amount"]))
            self.ui.table_orders.setItem(r, 3, QTableWidgetItem(d["time"]))
            self.ui.table_orders.setItem(r, 4, QTableWidgetItem(d["status"]))
        
        # lcd 주문 업데이트
        self.ui.lcd_num_orders.display(len(orders))
        waiting = [x for x in orders if x["status"] == "waiting"]
        self.ui.lcd_num_waiting.display(len(waiting))
        processing = [x for x in orders if x["status"] == "processing"]
        self.ui.lcd_num_processing.display(len(processing))
        delivery = [x for x in orders if x["status"] == "delivery"]
        self.ui.lcd_num_delivery.display(len(delivery))
        complete = [x for x in orders if x["status"] == "complete"]
        self.ui.lcd_num_complete.display(len(complete))
    
    def calc(self):
        menu_price = {
            "에스프레소" : 1000,
            "아메리카노" : 1500,
            "라떼" : 3000,
            "모카" : 3500,
            "초코 스무디" : 5500,
            "딸기 스무디" : 5500,
        }

        size_price = {
            "S": 0,
            "M": 500,
            "L": 1000,
            "XL": 1500,
        }

        price = 0
        for w in menu_widegets:
            menu = getattr(self.ui, w)
            if menu.isChecked():
                price = menu_price[menu.text()]
                break

        for w in size_widegets:
            size = getattr(self.ui, w)
            if size.isChecked():
                price += size_price[size.text()]
                break
        
        quantity = self.ui.spin_quantity.value()
        price = price * quantity

        self.ui.lb_order_amount.setText(f"총 주문 금액 : {price}원")
        return price
#...

## 주문 넣기

- 수량 변경 시 업데이트
- 수량 조절 스크롤, 다이얼 연동
  - `quantity_lock`으로 재귀상태 회피
- 발주 버튼을 누르면 주문내역으로 이동
  - 이전 내용까지 모두 불러오는 문제


### 실습코드

In [ ]:
class MainWindow(QMainWindow):
    quantity_lock = False

    def __init__(self):
        super(MainWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)

        self.timer = QTimer()
        self.timer.setInterval(990)
        self.timer.timeout.connect(self.tick)
        self.timer.start()

        self.load()

        for w in menu_widegets + size_widegets:
            widget = getattr(self.ui, w)
            widget.clicked.connect(self.calc)

        self.calc()

        self.ui.spin_quantity.valueChanged.connect(self.sync_quantity)
        self.ui.hs_quantity.valueChanged.connect(self.sync_quantity)
        self.ui.vs_quantity.valueChanged.connect(self.sync_quantity)
        self.ui.dial_quantity.valueChanged.connect(self.sync_quantity)

        self.ui.btn_order.clicked.connect(self.order)

# tick, load, calc 함수 생략...


    def sync_quantity(self):
        if self.quantity_lock:
            return
        
        self.quantity_lock = True

        spin = self.ui.spin_quantity
        hs = self.ui.hs_quantity
        vs = self.ui.vs_quantity
        dial = self.ui.dial_quantity

        values = [spin.value(), hs.value(), vs.value(), dial.value()]
        dup = {}
        target = 0
        for v in values:
            dup[v] = dup.get(v, 0) + 1
        for k, v in dup.items():
            if v == 1:
                target = k
        
        spin.setValue(target)
        hs.setValue(target)
        vs.setValue(target)
        dial.setValue(target)

        self.calc()

        self.quantity_lock = False
    
    def order(self):
        menu = ""
        for w in menu_widegets:
            radio = getattr(self.ui, w)
            if radio.isChecked():
                menu = radio.text()
            
        for w in size_widegets:
            radio = getattr(self.ui, w)
            if radio.isChecked():
                menu += " " + radio.text()
                break
        
        quantity = self.ui.spin_quantity.value()

        order = {
            "menu": menu,
            "quantity": str(quantity),
            "order_amount": self.calc(),
            "time": str(datetime.datetime.now()),
            "status": "waiting",
        }
        
        orders.append(order)

        mb = QMessageBox()
        mb.setText("발주되었습니다")
        mb.exec()

        self.load()

# ...

## 주문 상태 변경

- 주문내역 상태 변경 (대기중 -> 완료)

In [ ]:
class MainWindow(QMainWindow):
    quantity_lock = False

    def __init__(self):
      #...
      self.ui.table_orders.cellDoubleClicked.connect(self.change_order)
    
    #...
    def change_order(self, r, c):

        status = ["waiting", "processing", "delivery", "complete"]
        orders[r]["status"] = status[status.index(orders[r]["status"]) + 1]
        
        self.load()

#...